# **Exploração e análise de dados de crédito com SQL**

## Os dados: 

Os dados representam informações de clientes de um banco e contam com as seguintes colunas: 

* idade = idade do cliente
* sexo = sexo do cliente (F ou M)
* dependentes = número de dependentes do cliente
* escolaridade = nível de escolaridade do clientes
* estado_civil = estado civil dos clientes
* salario_anual = faixa salarial do cliente
* tipo_cartao = tipo de cartao do cliente
* meses_de_relacionamento = quantidade de meses do cliente desde o seu cadastro
* qtd_produtos = quantidade de produtos comprados nos últimos 12 meses
* iteracoes_12m = quantidade de iterações/transacoes nos ultimos 12 meses
* meses_inativo_12m = quantidade de meses que o cliente ficou inativo
* limite_credito = limite de credito do cliente
* valor_transacoes_12m = valor das transações dos ultimos 12 meses
* qtd_transacoes_12m  = quantidade de transacoes dos ultimos 12 meses

A tabela foi criada no **AWS Athena** junto com o **S3 Bucket** com uma versão dos dados disponibilizados em: https://github.com/andre-marcos-perez/ebac-course-utils/tree/main/dataset

Foi também utilizado o pacote o pacote pandas no para fazer a conversão da virgulas em ponto nas colunas de valores monetários, garantindo que o AWS entenda corretamente os dados do dataset.

In [1]:
import pandas as pd

# Leitura do arquivo, convertendo os valores com virgula para valores com ponto.
teste = pd.read_excel('credito1.xlsx')
# Remoção das colunas que não serão utilizadas para o estudo.
teste = teste.drop(['id','default'],axis=1)
# Geração do arquivo no formato csv sem o index e o header, separando as colunas com a virgula
teste.to_csv('credito1.csv',sep=',',index=None,header=None)

## **Exploração de dados:**

Antes de mais nada, é importante ter um entendimento sobre o dataset e suas caracteristicas, para uma melhor análise posterior.

**Qual a quantidade de informações temos na nossa base de dados?**

Query: SELECT count(*) FROM credito
> Resposta: 10127 linhas

**Como são os dados** 

Query: SELECT * FROM credito LIMIT 10;

![Dez primeiras linhas do dataset](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/0ae0d2ce516bd26e6d8fbfa9c713c3a54f14386c/results/limite10.png)

> É possível reparar que existem algumas informações nulas na tabela (valor na), vamos olhar mais de perto para os valores de cada coluna! 


**Quais os tipos de cada coluna?**

**Query:** DESCRIBE credito

![Descrição dos tipos de dados](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/describe.jpg)

Agora que ja entendemos quais são os tipos de dados, vamos olhar mais atentamente para as varíaveis que não são numéricas.

**Quais são os tipos de escolaridade disponíveis no dataset?**

**Query:** SELECT DISTINCT escolaridade FROM credito

![Tipos de escolaridade](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/escolaridade.png)


> Os dados contam com vários níveis de escolaridade e é possível perceber que temos valores nulos (na) no dataset, trataremos isso mais tarde!

Será que temos mais valores nulos?

**Quais são os tipos de estado_civil disponíveis no dataset?**

**Query:** SELECT DISTINCT estado_civil  FROM credito

![Tipos de estado civil](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/estadocivil.png)

> Novamente encontramos valores nulos nos dados de estado civil!

**Quais são os tipos de salario_anual disponíveis no dataset?**

**Query:** SELECT DISTINCT salario_anual  FROM credito

![Tipos de salario anual](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/salarioanual.png)

> Os salários nesse dataset não estão expostas com o valor exato que o cliente ganha. O que está informado é a faixa salarial de cada um. Também contem dados nulos. 

**Quais são os tipos de cartão disponíveis no dataset?**

**Query:** SELECT DISTINCT tipo_cartao  FROM credito

![Tipos de Cartão](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/tipocartao.png)

> Aqui vemos que não há necessidade de tratar valores nulos.

## **Análise de dados**

Uma vez que exploramos o dados e buscamos entender quais são as informações que temos no nosso banco de dados, podemos analisar as informações para buscar entender o que está acontecendo no banco de dados. Vamos fazer perguntas:

**Nesse banco de dados, quantos clientes temos por cada faixa salarial?**

**Query:** select count(*) as quantidade, salario_anual from credito group by salario_anual order by quantidade desc;

![Quantidade para cada faixa salarial](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/qtdeporsalario.png)

Aproveitar os dados para gerar um gráfico, facilita a interpretação dos dados.

![Gráfico da quantidade x faixa salarial](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/qtdeporsalarioplot.png)



> Pelo menos 35% dos clientes possuem uma renda menor que 40K e existem 1112 clientes que não informaram ou não consta a faixa salarial. 

> Pensando no maior público, pode ser interessante para a empresa focar nesse público de mais baixa renda. 

**Nesse banco de dados, quantos clientes são homens e quantos são mulheres?**

**Query:** select count(*) as Quantidade, sexo from credito group by sexo


![Quantidade para cada sexo](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/qtdeporsexo.png)

Utilizando do excel para gerar novamente os gráficos:

![Quantidade para cada sexo - gráfico](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/qtdeporsexoplot.png)

> A maioria dos clientes desse banco é do sexo feminino. Porém, a proporção está bem equilibrada.

**Queremos focar o nosso marketing de maneira adequada para nossos clientes, qual será a idade deles?**

**Query:** select avg(idade) as media_idade, min(idade) as min_idade, max(idade) as max_idade, sexo from credito group by sexo

![Média de idades por sexo](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/mediaidade.png)

> Por meio dessa análise não foi possível extrair nenhuma informação relevante.  A menor idade dos dois sexos é a mesma e a média é muito similar. A unica  diferença é a idade máxima. mas isso não foi delimitante para interferir substancialmente no resultado.

**Qual a maior, menor e a média de transação dos clientes de acordo com a faixa salarial?**

**Query:** select min(valor_transacoes_12m) as transacao_minima, max(valor_transacoes_12m) as transacao_maxima,
avg(valor_transacoes_12m) as transacao_media, "salario_anual" from credito group by "salario_anual" order by transacao_media desc;

![Valor transacoes](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/transacaoporsalario.png)

> São curiosos esses dados, pois por mais que sejam faixas salarias bem distintas, os valores das transações se encontram bem parecidos ou sem tanta discrepância aparente. Isso mostra como todos os grupos mantém um gasto elevado de toda forma.

**Quais as características dos clientes que possuem os maiores creditos?**

**Query:** 
select max(limite_credito) as limite_credito, escolaridade, tipo_cartao, sexo, estado_civil 
from credito 
where escolaridade != 'na' and tipo_cartao != 'na' and estado_civil != 'na' 
group by escolaridade, tipo_cartao, sexo, estado_civil 
order by limite_credito 
desc limit 10

![Valor limite máximo](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/limitemax.png)

> Não parece haver um impacto da escolaridade no limite. O limite mais alto é oferecido para um homem sem educação formal. O cartão também parece não estar relacionado com a escolaridade nem com o limite e nem com o estado civil. Mas, a tendência é a dos homens terem um maior limite (9 dos 10 resultados são de homens)


**Quais as características dos clientes que possuem os menores creditos?**

**Query:** 
select max(limite_credito) as limite_credito, escolaridade, tipo_cartao, sexo, estado_civil 
from credito 
where escolaridade != 'na' and tipo_cartao != 'na' and estado_civil != 'na' 
group by escolaridade, tipo_cartao, sexo, estado_civil 
order by limite_credito asc limit 10;

![Valor limite](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/limitemin.png)

> Os 4 menores limites são de mulheres com doutorado, o que talvez mostre que um alto grau acadêmico não garante um limite alto. Também foi possível perceber que todos os 10 menores limites são de mulheres enquanto os maiores limites predomina os homens. Metade das mulheres nesse recorte dos dados também estão divorciadas.

**Existe alguma relação entre os meses de relacionamento do cliente com a empresa e o limite do banco?**

**Query:** 
SELECT avg(limite_credito) as media_limite, faixa_meses, sexo
FROM (
  SELECT *, 
    CASE
      WHEN "meses_de_relacionamento" >= 1 AND "meses_de_relacionamento" <= 10 THEN '1-10'
      WHEN "meses_de_relacionamento" >= 11 AND "meses_de_relacionamento" <= 20 THEN '11-20'
      WHEN "meses_de_relacionamento" >= 21 AND "meses_de_relacionamento" <= 30 THEN '21-30'
      WHEN "meses_de_relacionamento" >= 31 AND "meses_de_relacionamento" <= 40 THEN '31-40'
      WHEN "meses_de_relacionamento" >= 41 AND "meses_de_relacionamento" <= 50 THEN '41-50'
      WHEN "meses_de_relacionamento" >= 51 AND "meses_de_relacionamento" <= 60 THEN '51-60'      
      ELSE 'others'
    END AS faixa_meses
  FROM credito
) AS subquery
GROUP BY faixa_meses, sexo
order by media_limite desc; 

![Media do limite por Meses Relacionamento](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/avglimitepormeses.png)

![Media do limite por Meses Relacionamento - Gráfico](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/avglimitepormesesplot.png)

>É possivel perceber que não há uma relação direta da média do limite para mulheres de acordo com os meses como cliente. Mas, para os homens, existe uma tendência de que o limite seja maior entre 31-40 meses, fazendo uma curva ao centro.


**Será que as mulheres gastam mais?**

**Query:** 
select max(valor_transacoes_12m) as maior_valor_gasto, avg(valor_transacoes_12m) as  media_valor_gasto, min(valor_transacoes_12m) as min_valor_gasto, sexo 
from credito
group by sexo 


![Valor transacoes/sexo](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/valortransacaoporsexo.png)

> Apesar da diferença nos limites, os gastos de homens e mulheres são similares!

Por fim, 

**O salário impacta no limite?**

**Query:** select count(*) as Qtde_clientes, avg(valor_transacoes_12m) as media_valor_transacoes, avg(limite_credito) as media_limite,  sexo,   salario_anual from credito 
where salario_anual != 'na'
group by sexo, salario_anual
order by media_limite desc

![Valor salario_anualLimite](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/limiteporsalario.png)

>Como esperado, as pessoas que tem menor faixa salarial também apresentam menor limite de credito. Mas, e se fizermos o comparativo entre clientes ou média do limite com a faixa salarial?

![limite por faixa salarial](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/limiteporsalarioplot.png)

>Os menores limites salariais estão com as mulheres, assim como as mulheres se encontram na menores faixas salariais.

![Clientes por faixa salarial](https://raw.githubusercontent.com/Caiombr/credit_exp_e_analise/main/results/clientesporsalarioplot.png)

>As mulheres se encontrar mais presentes na faixa abaixo de $40k, apesar de serem a maior quantidade de clientes da empresa.


# Conclusão

Essas foram **algumas** análises possiveis extraídas do dataset de crédito.  

Alguns insights que podem ser trazidos aqui:

- a maior parte dos clientes possui renda até 40K, em torno de 35%
- a faixa salarial dos clientes homens é maior, mesmo que tenha mais mulheres como clientes
- a escolaridade não parece influenciar no limite nem no tipo do cartão
- os clientes com maiores limites são em sua maioria homens 
- os clientes com menores limites são em sua maioria mulheres, inclusive mulheres com alto grau acadêmico
- o limite médio das mulheres se mantém constante, mesmo ao passar de muitos meses de relacionamento
- a faixa salarial impacta diretamente no limite de crédito
- nao existem clientes com salário anual acima de 60K do sexo feminino

**Uma exploração maior dos dados pode explicar porque as mulheres tem menor crédito. Isso também pode ser um problema cultural que pode ser repensado!**